# 股债收益差

In [1]:
# 1.获取数据，需指数收盘点位、pe-ttm、股息率、pb-ttm； 中债10年国债收益率
# 2.筛选指数
# 3.判断指数分布
# 4.计算股债收益差
# 5.回测收益
# 6.判断是否正太分布最为有效，是否需要使用其他分布
# 7.判断那种指标更有效
# 8.获取当前分位点，给出推荐建议

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [3]:
# 通过akshare获取指数数据
import akshare as ak

# 获取指数数据

* 大盘指数

In [2]:
# PE数据
stock_market_pe_lg_df = ak.stock_index_pe_lg(symbol="沪深300").sort_index(ascending=False)
# PB数据
stock_market_pb_lg_df = ak.stock_index_pb_lg(symbol="沪深300").sort_index(ascending=False)
# 股息率数据 choice of {"上证A股", "深证A股", "创业板", "科创板"}
stock_a_gxl_lg_df = ak.stock_a_gxl_lg(symbol="上证A股").sort_index(ascending=False)

* 获取大盘指数数据

In [4]:
fund_info_index_em_df = ak.fund_info_index_em(symbol="行业主题", indicator="被动指数型")

In [5]:
fund_info_index_em_df

,基金代码,基金名称,单位净值,日期,日增长率,近1周,近1月,近3月,近6月,近1年,近2年,近3年,今年来,成立来,手续费,起购金额,跟踪标的,跟踪方式
0,012768,华夏中证动漫游戏ETF联接A,1.4335,2023-04-14,-1.02,0.66,45.81,64.73,90.27,61.79,NaN,NaN,74.33,43.35,0.12,10元,行业主题,被动指数型
1,012769,华夏中证动漫游戏ETF联接C,1.4267,2023-04-14,-1.02,0.66,45.78,64.63,90.00,61.32,NaN,NaN,74.18,42.67,0.00,10元,行业主题,被动指数型
2,012728,国泰中证动漫游戏ETF联接A,1.4104,2023-04-14,-0.91,0.94,42.62,60.91,85.09,59.17,NaN,NaN,70.09,41.04,0.10,10元,行业主题,被动指数型
3,012729,国泰中证动漫游戏ETF联接C,1.4028,2023-04-14,-0.92,0.93,42.59,60.78,84.80,58.69,NaN,NaN,69.93,40.28,0.00,10元,行业主题,被动指数型
4,161628,融通中证云计算与大数据主题指数(LOF)A,1.2946,2023-04-14,0.61,-4.87,16.73,40.75,63.46,58.57,19.07,50.53,46.96,-4.02,0.12,10元,行业主题,被动指数型
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,018173,华泰柏瑞中证全指电力ETF发起式联接C,1.0000,2023-04-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,10元,行业主题,被动指数型
720,017832,华夏中证港股通消费主题ETF发起式联接A,0.9932,2023-04-14,0.18,-1.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.68,0.12,10元,行业主题,被动指数型
721,017833,华夏中证港股通消费主题ETF发起式联接C,0.9930,2023-04-14,0.18,-1.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.70,0.00,10元,行业主题,被动指数型
722,018172,华泰柏瑞中证全指电力ETF发起式联接A,1.0000,2023-04-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.12,10元,行业主题,被动指数型


In [30]:
fund_exchange_rank_em_df = ak.fund_exchange_rank_em()
ETF_name = fund_exchange_rank_em_df['基金简称']

In [32]:
# ETF_name中同一类ETF只保留一种



In [31]:
ETF_name

0         华夏中证动漫游戏ETF
1       华泰柏瑞中证动漫游戏ETF
2         国泰中证动漫游戏ETF
3          华夏中证云计算ETF
4          招商中证云计算ETF
            ...      
751    招商中证500增强策略ETF
752     天弘中证1000增强ETF
753       华夏中证全指运输ETF
754    鹏华国证疫苗与生物科技ETF
755       富国中证绿色电力ETF
Name: 基金简称, Length: 756, dtype: object

* 申万行业指数

In [12]:
#index_value_hist_funddb_df = ak.index_value_hist_funddb(symbol="大盘成长", indicator="市盈率")
# 获取行业指数名称
index_value_name_funddb_df = ak.index_value_name_funddb()
index_value_name = index_value_name_funddb_df[index_value_name_funddb_df['指数名称'].str.contains('申万')]['指数名称'].unique()
index_value_hist_funddb_df = ak.index_value_hist_funddb(symbol=index_value_name[0], indicator="市净率") #choice of {'市盈率', '市净率', '股息率', '风险溢价'}

In [7]:
index_value_name_funddb_df = ak.index_value_name_funddb()
print(index_value_name_funddb_df)

            指数名称   最新PE    PE分位  最新PB   PB分位   股息率  股息率分位        指数代码  \
0           光伏产业  19.53    0.21  3.85  50.05  0.67  67.05  931151.CSI   
1           中证新能  18.78    0.31  3.40  57.38  0.61  46.70   399808.SZ   
2          中证新能源  18.78    0.34  3.40  63.87  0.61  36.30  930771.CSI   
3          CS稀金属  11.85    0.79  3.37  28.12  0.60  69.78  930632.CSI   
4          胡志明指数  10.74    1.00  0.00   1.92  1.92  33.14  VNINDEX.GI   
..           ...    ...     ...   ...    ...   ...    ...         ...   
254      500价值稳健  16.16  100.00  1.13  55.60  3.48  61.99  931587.CSI   
255  富时发达市场REITs   0.00    0.00  0.00   0.00  0.00   0.00    ERGLU.FI   
256  富时中国A-H50指数   0.00    0.00  0.00   0.00  0.00   0.00   FCAH50.FI   
257    万得可转债等权指数   0.00    0.00  0.00   0.00  0.00   0.00   889033.WI   
258         中证畜牧   0.00   97.55  0.00  26.60  0.00  79.29  930707.CSI   

         指数开始时间        更新时间  
0    2019-04-22  2023-04-14  
1    2015-02-10  2023-04-14  
2    2015-12-16  2023-04-14  
3  

In [3]:
sw_index_first_info_df = ak.sw_index_first_info()
print(sw_index_first_info_df)

         行业代码  行业名称  成份个数  静态市盈率  TTM(滚动)市盈率   市净率  静态股息率
0   801010.SI  农林牧渔    99  45.97       45.31  2.74   0.63
1   801030.SI  基础化工   347  15.25       15.27  2.31   2.20
2   801040.SI    钢铁    45   8.69       13.56  1.02   5.47
3   801050.SI  有色金属   129  16.16       15.19  2.68   1.17
4   801080.SI    电子   309  33.69       33.04  3.22   1.07
5   801880.SI    汽车   246  24.97       25.85  2.10   1.39
6   801110.SI  家用电器    79  16.57       15.56  2.75   3.05
7   801120.SI  食品饮料   120  34.96       33.22  6.98   1.93
8   801130.SI  纺织服饰   113  17.07       17.96  1.86   3.56
9   801140.SI  轻工制造   149  22.26       23.93  2.29   1.87
10  801150.SI  医药生物   363  26.86       25.75  3.46   1.11
11  801160.SI  公用事业   123  21.56       21.19  1.88   2.21
12  801170.SI  交通运输   125   9.37        9.44  1.30   4.50
13  801180.SI   房地产   115  12.47       13.30  0.97   3.51
14  801200.SI  商贸零售   103  28.98       29.19  2.34   1.69
15  801210.SI  社会服务    73  63.24       62.53  3.57   0.54
16  801780.SI 

In [33]:
index_value_name

array(['有色金属(申万)', '煤炭(申万)', '光伏设备(申万)', '电力设备(申万)', '国防军工(申万)',
       '生物制品(申万)', '医药生物(申万)', '交通运输(申万)', '银行(申万)', '基础化工(申万)',
       '石油石化(申万)', '中盘指数(申万)', '通信设备(申万)', '通信(申万)', '环保(申万)', '化学制药(申万)',
       '家用电器(申万)', '风电设备(申万)', '纺织服饰(申万)', '电子(申万)', '保险Ⅱ(申万)',
       '小盘指数(申万)', '券商Ⅱ(申万)', '建筑装饰(申万)', '轻工制造(申万)', '综合(申万)',
       '建筑材料(申万)', '机械设备(申万)', '非银金融(申万)', '钢铁(申万)', '传媒(申万)', '大盘指数(申万)',
       '食品饮料(申万)', '房地产(申万)', '公用事业(申万)', '计算机(申万)', '商贸零售(申万)', '汽车(申万)',
       '农林牧渔(申万)', '社会服务(申万)', '美容护理(申万)'], dtype=object)

# 获取中债10年国债收益率

In [17]:
bond_zh_10_rate_df = ak.bond_zh_us_rate()[['日期','中国国债收益率10年']].dropna().reset_index(drop=True)
bond_zh_10_rate_df

,日期,中国国债收益率10年
0,2002-01-04,3.2096
1,2002-01-07,3.2003
2,2002-01-08,3.5225
3,2002-01-09,3.5896
4,2002-01-10,3.5784
...,...,...
5311,2023-04-10,2.8455
5312,2023-04-11,2.8203
5313,2023-04-12,2.8158
5314,2023-04-13,2.8277
